In [0]:
# Import the necessary Libraries
import torch
import pandas as pd
from torchvision import transforms, utils
from torch.autograd import Variable
import torch.utils.data as data_utils
import numpy as np
import matplotlib.pyplot as graph
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset
iris_df = pd.read_csv("iris.csv")
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# Convert the categorical label to Numerical
# Define the Label Encoder
le = preprocessing.LabelEncoder()
labels = list(iris_df["Species"].unique())
le.fit(labels)
iris_df["Species"] = le.transform(iris_df["Species"])
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [0]:
# Seperate the features and labels seperate
iris_features = iris_df[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]

iris_labels = iris_df[["Species"]]

In [0]:
# Split into train and test
train_feat , test_feat, train_label, test_label = train_test_split(iris_features, iris_labels, test_size=0.20, random_state=42)

In [0]:
# Convert the train into tensors and load as Dataloaders
train_feat = torch.Tensor(train_feat.values)

train_label = torch.LongTensor(train_label.values)

train_data = data_utils.TensorDataset(train_feat, train_label)

train_loader = data_utils.DataLoader(dataset=train_data,batch_size=32,shuffle=True)

In [0]:
# Convert the test into tensors and load as Dataloaders
test_feat = torch.Tensor(test_feat.values)

test_label = torch.LongTensor(test_label.values)

test_data = data_utils.TensorDataset(test_feat,test_label)

test_loader = data_utils.DataLoader(dataset=test_data,batch_size=8,shuffle=True)

In [0]:
# Define the neural Network
class IrisNet(torch.nn.Module):
    def __init__(self,input_layer,hidden_1_size,hidden_2_size,output_layer):
        super(IrisNet,self).__init__()
        self.first_layer=torch.nn.Linear(input_layer,hidden_1_size)
        self.first_layer_activation=torch.nn.ReLU()
        self.second_layer=torch.nn.Linear(hidden_1_size,hidden_2_size)
        self.second_layer_activation=torch.nn.ReLU()
        self.third_layer=torch.nn.Linear(hidden_2_size,output_layer)
        self.third_layer_activation=torch.nn.Sigmoid()

    def forward(self,x):
        output_1=self.first_layer(x)
        output_1=self.first_layer_activation(output_1)
        output_2=self.second_layer(output_1)
        output_2=self.second_layer_activation(output_2)
        output_3=self.third_layer(output_2)
        return output_3

In [0]:
# Declare the model and its hyperparameters
#Specifying the no of neurons in every layer
model = IrisNet(4,100,40,3)

#Defining the loss function for the model
criterion = torch.nn.CrossEntropyLoss()

#defining the learning rate
learning_rate = 0.001

#defining the optimizer 
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)


In [0]:
# Some utility variables
no_of_epoch = 20

training_loss=[]
training_accuracy=[]
testing_loss=[]
testing_accuracy=[]
epoch_list=[]

In [12]:
for epoch in range(no_of_epoch):
    
    total_trained = 0
    correctly_trained = 0
    
    for i ,(train_features,train_labels) in enumerate(train_loader):
        # Setting up the input 
        train_features=Variable(train_features)
        train_labels=Variable(train_labels)
        train_labels = train_labels.reshape(-1)
        
        # Setting model for training
        model.train()
        
        # Initializing gradients to zero 
        optimizer.zero_grad()
        
        # Predicting the output
        output = model(train_features)
        
        
        # Calculating the loss
        loss = criterion(output,train_labels)
        
        # Backpropogation
        loss.backward()
        
        # Optimizing the weights
        optimizer.step()
        
        # Getting the prediciton labels
        total_trained += train_labels.size(0)
        
        _ , predicted_labels = torch.max(output.data,1)
   
        correctly_trained += (train_labels.data==predicted_labels).sum().item()
    
    #Printing the Accuracy and loss values for training data
    print("Epoch:%d"%(epoch))
    print("Training Accuracy:%f loss:%f"%((correctly_trained/total_trained)*100,loss))
    
    # Appending the current epoch details
    training_loss.append(loss.data)
    training_accuracy.append((correctly_trained/total_trained)*100)
    epoch_list.append(epoch)
    
    # Now testing with the test data
    output=model(Variable(test_feat))
    test_label = Variable(test_label)
    test_label = test_label.reshape(-1)
    loss=criterion(output,Variable(test_label))
    
    _ , predicted_label = torch.max(output.data,1)
    
    correct_prediction=(predicted_label==test_label).sum().item()
    
    #Prinitng the Accuracy and loss values for testing data
    print("Testing Accuracy:%f loss:%f\n"%((correct_prediction/test_label.size(0))*100,loss))
    
    testing_loss.append(loss.data)
    testing_accuracy.append((correct_prediction/test_label.size(0))*100)

Epoch:0
Training Accuracy:53.333333 loss:0.981631
Testing Accuracy:56.666667 loss:0.967827

Epoch:1
Training Accuracy:64.166667 loss:0.970191
Testing Accuracy:56.666667 loss:0.963780

Epoch:2
Training Accuracy:69.166667 loss:0.962514
Testing Accuracy:60.000000 loss:0.959757

Epoch:3
Training Accuracy:74.166667 loss:0.964031
Testing Accuracy:63.333333 loss:0.955696

Epoch:4
Training Accuracy:75.000000 loss:0.963386
Testing Accuracy:76.666667 loss:0.951860

Epoch:5
Training Accuracy:76.666667 loss:0.941888
Testing Accuracy:70.000000 loss:0.947737

Epoch:6
Training Accuracy:76.666667 loss:0.942402
Testing Accuracy:73.333333 loss:0.943743

Epoch:7
Training Accuracy:80.000000 loss:0.949919
Testing Accuracy:76.666667 loss:0.940195

Epoch:8
Training Accuracy:79.166667 loss:0.931396
Testing Accuracy:76.666667 loss:0.936401

Epoch:9
Training Accuracy:80.000000 loss:0.930449
Testing Accuracy:76.666667 loss:0.932594

Epoch:10
Training Accuracy:71.666667 loss:0.951581
Testing Accuracy:76.666667 lo